<a href="https://colab.research.google.com/github/eunzzae/PJT_C.lab/blob/main/RS_Project_TalkD_d5_240129.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Load

In [ ]:
import pandas as pd
import numpy as np

customer_df = pd.read_csv('Data_1_Simulated_Customer_Data_50k_v2_240117_modi.csv')
course_df = pd.read_csv('Data_2_TalkD_coursemap_v2_240119_modi.csv')
scores_df = pd.read_csv('Data_2_TalkD_coursemap_Score_240119_modi.csv')


In [ ]:
course_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 18 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   일련번호         32 non-null     object
 1   주제    구분     32 non-null     object
 2   내용구분         32 non-null     object
 3   시기           32 non-null     object
 4   시기_2         32 non-null     object
 5   코스 명         32 non-null     object
 6   3T           32 non-null     int64 
 7   NVC          32 non-null     int64 
 8   육아정보         32 non-null     int64 
 9   양육자Only      6 non-null      object
 10  추천태그         32 non-null     object
 11  관련 육아고민      32 non-null     object
 12  세부내용         15 non-null     object
 13  기간(week)     32 non-null     int64 
 14  Score_3T     32 non-null     int64 
 15  Score_NVC    32 non-null     int64 
 16  Score_육아정보   32 non-null     int64 
 17  Total_Score  32 non-null     int64 
dtypes: int64(8), object(10)
memory usage: 4.6+ KB


In [ ]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customer_id             50000 non-null  object 
 1   Name                    50000 non-null  object 
 2   Gender                  50000 non-null  object 
 3   Age                     50000 non-null  int64  
 4   Number_of_Children      50000 non-null  int64  
 5   Child_1_AgeInMonths     50000 non-null  int64  
 6   Child_1_Gender          50000 non-null  object 
 7   Child_2_AgeInMonths     14957 non-null  float64
 8   Child_2_Gender          14957 non-null  object 
 9   Child_3_AgeInMonths     2474 non-null   float64
 10  Child_3_Gender          2474 non-null   object 
 11  Parenting_Concerns      50000 non-null  object 
 12  Parenting_Stress_Level  50000 non-null  int64  
dtypes: float64(2), int64(4), object(7)
memory usage: 5.0+ MB


# Rule based Model

##### Data preprocessing (월령주기 split)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 아이 월령 처리
course_df[['Start_Month', 'End_Month']] = course_df['시기_2'].str.split('-', expand=True)
course_df['Start_Month'] = pd.to_numeric(course_df['Start_Month'])
course_df['End_Month'] = pd.to_numeric(course_df['End_Month'])

# 가중치 추가
unique_content_types = course_df['내용구분'].unique()
content_type_weights = {
    '필수': 5,
    '발달': 4,
    '생활': 3,
    '공감': 2,
    '훈육': 1
}
course_df['Content_Weight'] = course_df['내용구분'].map(content_type_weights)

# 고객의 고민과 고민 리스트의 일치도 여부 확인
def calculate_weighted_match_percentage(concerns_list, course_concerns):
    course_concerns_set = set(course_concerns.split(", "))
    weighted_match_count = 0
    total_weight = 0
    for idx, concern in enumerate(concerns_list):
        if concern in course_concerns_set:
            weight = len(concerns_list) - idx
            weighted_match_count += weight
        total_weight += len(concerns_list) - idx
    return (weighted_match_count / total_weight) * 100 if total_weight > 0 else 0

# 모든 코스에 대한 점수 계산
def calculate_all_course_scores(scores_df, course_df):
    course_scores_list = []
    for concerns in course_df['관련 육아고민'].str.split(", "):
        course_scores = scores_df[scores_df['통합 육아고민 리스트'].isin(concerns)].iloc[:, 1:].sum().values
        course_scores_list.append(course_scores)
    return course_scores_list

# 고객별 유사도 점수 계산
def calculate_course_similarity_scores(customer_concerns, customer_scores, all_course_scores):
    cosine_similarities = cosine_similarity([customer_scores], all_course_scores)[0]
    return cosine_similarities

# 유사도 점수를 포함하여 각 코스의 추천
def calculate_course_recommendation_score(course_concerns, customer_concerns, similarity_scores, course_name, course_df):
    weighted_match_percentage = calculate_weighted_match_percentage(customer_concerns, course_concerns)
    course_index = course_df[course_df['코스 명'] == course_name].index[0]
    similarity_score = similarity_scores[course_index]
    content_weight = course_df.loc[course_index, 'Content_Weight']
    return (weighted_match_percentage + similarity_score) * content_weight

# 코스 추천 함수
def get_top_3_courses(customer_name, child_ages, concerns, course_df, all_course_scores):
    recommendations = []
    recommended_courses = set()
    for age in child_ages:
        if pd.notnull(age):
            matching_courses = course_df[(course_df['Start_Month'] <= age) & (course_df['End_Month'] >= age)]
            for _, course in matching_courses.iterrows():
                course_name = course['코스 명']
                if course_name in recommended_courses:
                    continue
                recommendation_score = calculate_course_recommendation_score(course['관련 육아고민'], concerns, all_course_scores, course_name, course_df)
                recommendations.append((course_name, recommendation_score))
                recommended_courses.add(course_name)
    recommendations.sort(key=lambda x: x[1], reverse=True)
    top_3_courses = recommendations[:3]
    return [customer_name] + [course[0] for course in top_3_courses]

# 고객 데이터 처리 및 최종 추천
all_course_scores = calculate_all_course_scores(scores_df, course_df)

top_3_courses_list = []
for index, customer in customer_df.iterrows():
    child_ages = [customer['Child_1_AgeInMonths'], customer['Child_2_AgeInMonths'], customer['Child_3_AgeInMonths']]
    concerns = customer['Parenting_Concerns'].split(", ")
    customer_scores = scores_df[scores_df['통합 육아고민 리스트'].isin(concerns)].iloc[:, 1:].sum().values
    similarity_scores = calculate_course_similarity_scores(concerns, customer_scores, all_course_scores)
    top_courses = get_top_3_courses(customer['Name'], child_ages, concerns, course_df, similarity_scores)
    top_3_courses_list.append(top_courses)

df_top_3_courses = pd.DataFrame(top_3_courses_list, columns=['Customer_Name', 'Recommended_Course1', 'Recommended_Course2', 'Recommended_Course3'])
df_top_3_courses.head()


# Matrix based Model

##### Preprocessing

In [ ]:
# 'Period_2'와 'Concerns'를 연결하여 course_data에 새 열 생성
course_df['Vectorization_Column'] = course_df['시기_2'] + ", " + course_df['관련 육아고민']
customer_df['Vectorization_Column'] = customer_df['Parenting_Concerns']

course_df.head(), customer_df.head()

(  일련번호 주제    구분 내용구분         시기   시기_2          코스 명  3T  NVC  육아정보 양육자Only  \
 0    1    Basic   필수  가입 직후/all   0-48           첫걸음   4    4     2     NaN   
 1    2    Basic   발달       ~24m   0-24  생애초기 (intro)   5    3     2     NaN   
 2    3    Basic   발달     22~27m  22-27           두돌    4    4     2     NaN   
 3    4    Basic   발달     34~39m  34-39           세돌    2    6     2     NaN   
 4    5    Basic   발달       45m~  45-48    네돌 (outro)   2    6     2     NaN   
 
    ...                              관련 육아고민                            세부내용  \
 0  ...                    육아관, 관계, 발달, 상호작용                (3T/NVC기초) + 육아관   
 1  ...    육아관, 육아방법, 상호작용, 관계, 발달, 부모역할, 수면  언어자극과 두뇌발달, 아이들 발화 이전의 상호작용 방법   
 2  ...      육아관, 육아방법, 상호작용, 발달, 감정표현, 부모역할        두돌 발달사항 체크와 해당기간 상호작용 방법   
 3  ...  육아관, 육아방법, 상호작용, 발달, 감정표현, 부모역할, 훈육        세돌 발달사항 체크와 해당기간 상호작용 방법   
 4  ...  육아관, 육아방법, 상호작용, 발달, 감정표현, 부모역할, 훈육        네돌 발달사항 체크와 해당기간 상호작용 방법   
 
   기간(week)  Score_3T  Score_NVC  Sc

##### Cosine Similarity

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# 메트릭스 모델에서 상위 3개 코스를 추출하는 함수
def get_top_3_courses_cosine(similarity_matrix, course_names, course_df, customer_df, all_course_scores, scores_df):
    top_3_courses_list = []
    for idx, customer_similarity in enumerate(similarity_matrix.T):
        customer_concerns = customer_df.iloc[idx]['Parenting_Concerns'].split(", ")
        course_recommendations = []
        recommended_courses = set()
        for course_idx, similarity_score in enumerate(customer_similarity):
            course_name = course_names[course_idx]
            if course_name in recommended_courses:
                continue

            course_concerns = course_df.iloc[course_idx]['관련 육아고민']
            weighted_match_percentage = calculate_weighted_match_percentage(customer_concerns, course_concerns)
            recommendation_score = (weighted_match_percentage + similarity_score) / 2
            course_recommendations.append((course_name, recommendation_score))
            recommended_courses.add(course_name)

        course_recommendations.sort(key=lambda x: x[1], reverse=True)
        top_3_courses = [course[0] for course in course_recommendations[:3]]
        top_3_courses_list.append(top_3_courses)
    return top_3_courses_list

# 모든 코스에 대한 점수 계산
def calculate_all_course_scores(scores_df, course_df):
    all_course_scores = []
    for _, row in course_df.iterrows():
        concerns = set(row['관련 육아고민'].split(", "))
        scores = scores_df[scores_df['통합 육아고민 리스트'].isin(concerns)].iloc[:, 1:].sum().values
        all_course_scores.append(scores)
    return np.array(all_course_scores)

# 모든 코스 점수 계산
all_course_scores = calculate_all_course_scores(scores_df, course_df)

# 고객별 유사도 점수 계산
customer_similarity_scores = []
for index, customer in customer_df.iterrows():
    customer_concerns = customer['Parenting_Concerns'].split(", ")
    customer_scores = scores_df[scores_df['통합 육아고민 리스트'].isin(customer_concerns)].iloc[:, 1:].sum().values
    similarity_scores = calculate_course_similarity_scores(customer_concerns, customer_scores, all_course_scores)
    customer_similarity_scores.append(similarity_scores)

# 벡터화 및 코사인 유사도 계산
vectorizer = CountVectorizer()
course_vectorized = vectorizer.fit_transform(course_df['Vectorization_Column'])
customer_vectorized = vectorizer.transform(customer_df['Vectorization_Column'])
cos_sim_count = cosine_similarity(course_vectorized, customer_vectorized)

# 코스 이름 리스트 추출
course_names = course_df['코스 명'].tolist()

# 각 고객별 상위 3개 코스 추출
top_3_courses_matrix = get_top_3_courses_cosine(cos_sim_count, course_names, course_df, customer_df, all_course_scores, scores_df)
top_3_courses_matrix


[['놀이터 (영아)', '놀이터 (유아)', '놀이 상호작용 (발화 전)'],
 ['육아번아웃 다루기', '육아 중 부부대화 (영아)', '육아 중 부부대화 (유아)'],
 ['생애초기 (intro)', '놀이터 (유아)', '세돌 '],
 ['거절하기와 거절듣기', '자조능력 (이유식)', '놀이터 (영아)'],
 ['생애초기 (intro)', '육아 중 부부대화 (영아)', '육아 중 부부대화 (유아)'],
 ['생애초기 (intro)', '놀이터 (영아)', '놀이터 (유아)'],
 ['두돌 ', '세돌 ', '네돌 (outro)'],
 ['등하원 기관 적응 (유치원 3세반)', '세돌 ', '네돌 (outro)'],
 ['놀이터 (영아)', '감정과 욕구 (발화 전)', '감정과 욕구 (발화 후)'],
 ['양육자 자기연결', '육아번아웃 다루기', '등하원 기관 적응 (유치원 3세반)'],
 ['거절하기와 거절듣기', '세돌 ', '네돌 (outro)'],
 ['등하원 기관 적응 (1세반)', '등하원 기관 적응 (2세반)', '생애초기 (intro)'],
 ['육아 중 부부대화 (영아)', '육아 중 부부대화 (유아)', '놀이터 (유아)'],
 ['세돌 ', '네돌 (outro)', '거절하기와 거절듣기'],
 ['놀이터 (영아)', '놀이터 (유아)', '놀이 상호작용 (발화 전)'],
 ['양육자 자기연결', '육아번아웃 다루기', '등하원 기관 적응 (유치원 3세반)'],
 ['세돌 ', '네돌 (outro)', '훈육 (두돌 전)'],
 ['감정과 욕구 (발화 전)', '감정과 욕구 (발화 후)', '거절하기와 거절듣기'],
 ['생애초기 (intro)', '미디어 입문', '미디어 갈등'],
 ['육아 중 부부대화 (영아)', '육아 중 부부대화 (유아)', '놀이터 (유아)'],
 ['거절하기와 거절듣기', '놀이터 (영아)', '놀이터 (유아)'],
 ['생애초기 (intro)', '등하원 기관 적응 (유치원 3세반)', '훈육 (두

# Hybrid Model

In [ ]:
# 룰베이스 모델에서 각 추천 코스의 일치도 점수 추출
rule_based_scores = []

for index, row in df_top_3_courses.iterrows():
    scores = []
    for course in row[1:]:
        course_match = course_df[course_df['코스 명'] == course]['관련 육아고민']
        if not course_match.empty:
            course_concerns = course_match.iloc[0]
            score = calculate_weighted_match_percentage(customer_df.iloc[index]['Parenting_Concerns'].split(", "), course_concerns)
        else:
            score = 0
        scores.append(score)
    rule_based_scores.append(scores)

# 메트릭스 모델에서 각 코스에 대한 점수 추출
matrix_based_scores_cosine = []
for i in range(len(customer_df)):
    customer_scores = cos_sim_count[:, i]
    top_3_indices = np.argsort(customer_scores)[-3:][::-1]
    top_3_scores = [customer_scores[index] for index in top_3_indices]
    matrix_based_scores_cosine.append(top_3_scores)

# 모델 가중치 부여
weights = {0: 3, 1: 2, 2: 1}
final_scores_weighted = []

for rb_scores, mb_scores in zip(rule_based_scores, matrix_based_scores_cosine):
    rb_ranks = np.argsort(rb_scores)[::-1]
    mb_ranks = np.argsort(mb_scores)[::-1]
    total_scores = [0] * len(rb_scores)

    for rank, course_index in enumerate(rb_ranks):
        total_scores[course_index] += weights.get(rank, 0)
    for rank, course_index in enumerate(mb_ranks):
        total_scores[course_index] += weights.get(rank, 0)
    final_scores_weighted.append(total_scores)

# 각 고객별로 최종 점수가 가장 높은 상위 3개 코스 선택
final_recommendations = []
for scores, courses in zip(final_scores_weighted, top_3_courses_matrix):
    sorted_indices = np.argsort(scores)[::-1]
    recommended_courses = [courses[index] for index in sorted_indices[:3]]
    final_recommendations.append(recommended_courses)


In [ ]:
# 고객 이름을 포함하여 최종 추천 결과 표시
final_recommendations_with_names = []
for customer_name, recommendations in zip(customer_df['Name'], final_recommendations):
    final_recommendations_with_names.append([customer_name] + recommendations)

# 최종 결과 확인
final_recommendations_with_names[:5]


[['조서연', '놀이터 (유아)', '놀이터 (영아)', '놀이 상호작용 (발화 전)'],
 ['최하은', '육아 중 부부대화 (영아)', '육아번아웃 다루기', '육아 중 부부대화 (유아)'],
 ['이현우', '놀이터 (유아)', '세돌 ', '생애초기 (intro)'],
 ['장예은', '거절하기와 거절듣기', '놀이터 (영아)', '자조능력 (이유식)'],
 ['최예은', '생애초기 (intro)', '육아 중 부부대화 (영아)', '육아 중 부부대화 (유아)']]